In [98]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder

In [99]:
df = pd.read_csv('dataset/Loan_default.csv',sep=',')
df

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255342,8C6S86ESGC,19,37979,210682,541,109,4,14.11,12,0.85,Bachelor's,Full-time,Married,No,No,Other,No,0
255343,98R4KDHNND,32,51953,189899,511,14,2,11.55,24,0.21,High School,Part-time,Divorced,No,No,Home,No,1
255344,XQK1UUUNGP,56,84820,208294,597,70,3,5.29,60,0.50,High School,Self-employed,Married,Yes,Yes,Auto,Yes,0
255345,JAO28CPL4H,42,85109,60575,809,40,1,20.90,48,0.44,High School,Part-time,Single,Yes,Yes,Other,No,0


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255347 entries, 0 to 255346
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   LoanID          255347 non-null  object 
 1   Age             255347 non-null  int64  
 2   Income          255347 non-null  int64  
 3   LoanAmount      255347 non-null  int64  
 4   CreditScore     255347 non-null  int64  
 5   MonthsEmployed  255347 non-null  int64  
 6   NumCreditLines  255347 non-null  int64  
 7   InterestRate    255347 non-null  float64
 8   LoanTerm        255347 non-null  int64  
 9   DTIRatio        255347 non-null  float64
 10  Education       255347 non-null  object 
 11  EmploymentType  255347 non-null  object 
 12  MaritalStatus   255347 non-null  object 
 13  HasMortgage     255347 non-null  object 
 14  HasDependents   255347 non-null  object 
 15  LoanPurpose     255347 non-null  object 
 16  HasCoSigner     255347 non-null  object 
 17  Default   

In [101]:
# encoding data
encorder = OrdinalEncoder()
for col in df.columns[10:17]:
    df[col] = encorder.fit_transform(df[[col]])

In [104]:
# drop ID
df = df.drop(['LoanID'], axis=1)

KeyError: "['LoanID'] not found in axis"

In [111]:
x,y = df.iloc[:,:16], df.iloc[:,16]

In [88]:
# normalisasi data 
from sklearn.preprocessing import StandardScaler

std = StandardScaler()

x = std.fit_transform(x)

In [89]:
x

array([[ 8.33989509e-01,  8.96928115e-02, -1.08683299e+00, ...,
         9.99463619e-01,  1.41535378e+00,  9.99784630e-01],
       [ 1.70122109e+00, -8.23020714e-01, -4.43088703e-02, ...,
        -1.00053667e+00,  1.41535378e+00,  9.99784630e-01],
       [ 1.66888295e-01,  4.38543784e-02,  2.27148729e-02, ...,
         9.99463619e-01, -1.41606345e+00, -1.00021542e+00],
       ...,
       [ 8.33989509e-01,  5.95616130e-02,  1.13939142e+00, ...,
         9.99463619e-01, -1.41606345e+00,  9.99784630e-01],
       [-9.99521902e-02,  6.69789183e-02, -9.45840328e-01, ...,
         9.99463619e-01,  1.41535378e+00, -1.00021542e+00],
       [ 1.23425024e+00, -1.54201168e+00, -1.54004788e+00, ...,
        -1.00053667e+00, -3.54832254e-04,  9.99784630e-01]])

In [112]:
# # use pca 
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
X = pca.fit_transform(x)



In [90]:
# under sampling penerapan 
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(x, y)


In [114]:
# over sampling penerapan 
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = SMOTE().fit_resample(X, y)

In [115]:
from sklearn.model_selection import train_test_split

#Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [109]:
# pca after split 
from sklearn.decomposition import PCA

pca = PCA(n_components=11)
X_train = pca.fit_transform(X_train)
X_test = pca.fit_transform(X_test)

In [116]:
from sklearn.metrics import accuracy_score, classification_report
#Creating an XGBoost classifier
model = xgb.XGBClassifier()

#Training the model on the training data
model.fit(X_train, y_train)

#Making predictions on the test set
predictions = model.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predictions))

c:\Users\alam\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Accuracy: 0.8591240390792885

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.97      0.87     45127
           1       0.96      0.75      0.84     45151

    accuracy                           0.86     90278
   macro avg       0.88      0.86      0.86     90278
weighted avg       0.88      0.86      0.86     90278

